In [1]:
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import os

import IPython.display as ipd

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,precision_recall_curve
from sklearn.metrics import precision_recall_fscore_support,f1_score,fbeta_score
from sklearn.metrics import make_scorer
from sklearn.metrics import classification_report
from sklearn.svm import SVC, NuSVC, LinearSVC
from sklearn.utils import shuffle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

import librosa
import librosa.display
import ast

import keras
plt.rcParams['figure.figsize'] = (17, 5)



Using TensorFlow backend.


In [2]:
df = pd.read_csv('data.csv')

In [3]:
df.head()

,filename,chroma_cqt,chroma_cens,chroma_stft,spectral_centroid,spectral_bandwidth,spectral_contrast,rolloff,zero_crossing_rate,tonnetz,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,126014,0.267008,0.171130,0.237383,699.229710,785.999971,25.676751,1183.751175,0.041712,0.092841,...,-27.326481,-20.607473,-12.659972,-13.275584,-12.988043,-15.687397,-12.339302,2.211766,2.147864,-5.011512
1,54496,0.274600,0.207362,0.277710,1833.074733,2091.124342,25.274995,3818.560243,0.093890,0.044640,...,-5.424025,6.843907,-6.247772,3.441898,-0.244973,4.090439,-1.278771,9.071951,-0.169724,0.161876
2,65893,0.458305,0.253559,0.412075,2767.487106,2741.112704,20.450162,5889.689386,0.111506,0.013648,...,-8.586940,6.649025,-2.908488,3.085722,-3.682684,2.957125,-2.243021,1.578797,-4.537785,1.052801
3,71158,0.531325,0.258276,0.424391,1444.225428,2060.084462,19.861984,3089.631317,0.046734,0.000029,...,5.303405,-0.030636,-1.089045,2.822888,-2.630777,1.943587,-7.131413,1.516889,-0.094114,-0.776687
4,57821,0.412456,0.239312,0.335196,2253.305912,2412.719760,22.327731,4858.028957,0.099870,0.024816,...,-8.056653,-5.495337,-8.891809,-2.783748,-6.777088,-3.736772,-11.871335,-0.826213,-2.934075,6.011378


In [4]:
df.shape

(7997, 30)

In [5]:
genres = pd.read_csv('data/fma_metadata/genrelist.csv')

In [6]:
genres

,Unnamed: 0,genre,testtrainval
0,NaN,NaN,NaN
1,track_id,NaN,NaN
2,2,Hip-Hop,training
3,5,Hip-Hop,training
4,10,Pop,training
...,...,...,...
7997,154308,Hip-Hop,test
7998,154309,Hip-Hop,test
7999,154413,Pop,training
8000,154414,Pop,training


In [7]:
genres.rename({'Unnamed: 0':'track_id'},axis=1,inplace=True)

In [8]:
genres = genres.drop(index=[0,1])

In [9]:
genres.head()

,track_id,genre,testtrainval
2,2,Hip-Hop,training
3,5,Hip-Hop,training
4,10,Pop,training
5,140,Folk,training
6,141,Folk,training


In [10]:
genres['track_id'] = genres['track_id'].astype('int64')

In [11]:
df.rename({'filename':'track_id'},axis=1,inplace=True)

In [12]:
df['track_id']

0       126014
1        54496
2        65893
3        71158
4        57821
         ...  
7992     43695
7993     68410
7994     51291
7995     96902
7996     75441
Name: track_id, Length: 7997, dtype: int64

In [13]:
data=pd.merge(df,genres,on='track_id')

In [14]:
data['genre']

0        Instrumental
1        Experimental
2                Folk
3             Hip-Hop
4             Hip-Hop
            ...      
7992    International
7993     Experimental
7994              Pop
7995     Experimental
7996     Instrumental
Name: genre, Length: 7997, dtype: object

In [15]:
data.set_index('track_id',inplace=True)

In [16]:
data.head()

,chroma_cqt,chroma_cens,chroma_stft,spectral_centroid,spectral_bandwidth,spectral_contrast,rolloff,zero_crossing_rate,tonnetz,mfcc1,...,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,genre,testtrainval
track_id,,,,,,,,,,,,,,,,,,,,,
126014,0.267008,0.171130,0.237383,699.229710,785.999971,25.676751,1183.751175,0.041712,0.092841,-352.161438,...,-12.659972,-13.275584,-12.988043,-15.687397,-12.339302,2.211766,2.147864,-5.011512,Instrumental,training
54496,0.274600,0.207362,0.277710,1833.074733,2091.124342,25.274995,3818.560243,0.093890,0.044640,-290.169464,...,-6.247772,3.441898,-0.244973,4.090439,-1.278771,9.071951,-0.169724,0.161876,Experimental,training
65893,0.458305,0.253559,0.412075,2767.487106,2741.112704,20.450162,5889.689386,0.111506,0.013648,-80.027184,...,-2.908488,3.085722,-3.682684,2.957125,-2.243021,1.578797,-4.537785,1.052801,Folk,training
71158,0.531325,0.258276,0.424391,1444.225428,2060.084462,19.861984,3089.631317,0.046734,0.000029,-123.564980,...,-1.089045,2.822888,-2.630777,1.943587,-7.131413,1.516889,-0.094114,-0.776687,Hip-Hop,training
57821,0.412456,0.239312,0.335196,2253.305912,2412.719760,22.327731,4858.028957,0.099870,0.024816,-71.340874,...,-8.891809,-2.783748,-6.777088,-3.736772,-11.871335,-0.826213,-2.934075,6.011378,Hip-Hop,training


In [17]:
data['testtrainval'].value_counts()

training      6397
validation     800
test           800
Name: testtrainval, dtype: int64

In [18]:
y = data[['genre','testtrainval']]

In [19]:
y.drop('testtrainval',axis=1,inplace=True)

In [20]:
X = data.drop(['genre','testtrainval'],axis=1)

In [21]:
X2 = X.iloc[:,9:29]

In [22]:
X2

,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
track_id,,,,,,,,,,,,,,,,,,,,
126014,-352.161438,147.268738,-6.516946,14.641133,29.060827,-2.149615,-9.781580,-8.744802,-11.623240,-17.436735,-27.326481,-20.607473,-12.659972,-13.275584,-12.988043,-15.687397,-12.339302,2.211766,2.147864,-5.011512
54496,-290.169464,107.796921,-17.293709,29.786123,8.302071,11.566362,-11.648953,-6.970734,-3.238187,-2.197775,-5.424025,6.843907,-6.247772,3.441898,-0.244973,4.090439,-1.278771,9.071951,-0.169724,0.161876
65893,-80.027184,61.457649,18.617395,38.681793,3.637755,21.321770,8.109901,4.545240,-0.697991,7.539818,-8.586940,6.649025,-2.908488,3.085722,-3.682684,2.957125,-2.243021,1.578797,-4.537785,1.052801
71158,-123.564980,127.734001,22.012423,42.193047,28.423532,-0.134702,7.428549,6.176642,-11.435475,4.944324,5.303405,-0.030636,-1.089045,2.822888,-2.630777,1.943587,-7.131413,1.516889,-0.094114,-0.776687
57821,-71.340874,92.360405,-3.542409,-0.463810,-13.841660,9.869255,-6.058197,0.128541,-7.656012,0.641387,-8.056653,-5.495337,-8.891809,-2.783748,-6.777088,-3.736772,-11.871335,-0.826213,-2.934075,6.011378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43695,-43.920574,101.167107,-54.982464,20.285261,-10.458629,19.034140,-2.695947,5.690491,-0.599940,2.894833,9.848370,17.061972,2.227302,-2.029427,-8.724811,2.876313,-2.736481,-2.547942,-10.971077,-4.219173
68410,-16.044903,70.045433,8.357967,28.988680,-3.816420,-5.503837,-0.557270,-0.938137,-1.681377,-6.005097,-4.045043,0.901013,-8.597396,-5.432930,-12.551827,-5.137374,-5.069657,1.975084,-0.862782,-1.661779
51291,-129.250885,63.081413,-90.161919,-19.360296,-11.701410,-1.094478,-11.671681,-0.070076,1.406723,9.443295,0.234382,4.351338,-6.455239,-0.917210,-7.075034,-6.199870,-16.941978,-10.509199,-13.883102,-8.334950


In [23]:
X_train, X_test, y_train , y_test = train_test_split(X,y,stratify=y,test_size=0.15)

In [24]:
X_train.shape,X_test.shape

((6797, 29), (1200, 29))

In [25]:
X_train

,chroma_cqt,chroma_cens,chroma_stft,spectral_centroid,spectral_bandwidth,spectral_contrast,rolloff,zero_crossing_rate,tonnetz,mfcc1,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
track_id,,,,,,,,,,,,,,,,,,,,,
121930,0.394664,0.237802,0.369295,1148.526073,1281.161639,22.253155,2268.252951,0.059018,0.026034,-141.503876,...,7.945023,7.871596,0.210456,-3.393200,1.676501,-2.562336,1.695775,-4.684369,-3.033432,2.229170
90828,0.509369,0.270168,0.406470,1513.017881,1734.716464,21.199411,3114.064503,0.078520,0.042119,-189.189728,...,1.585011,2.184873,-4.988668,0.016117,-7.461729,1.227588,-9.956388,-2.851762,-2.814186,-2.234981
139532,0.244814,0.178490,0.222107,819.418010,770.688741,28.238665,1380.516652,0.052048,-0.015968,-286.174255,...,-9.457005,-13.915222,-10.770562,-4.405025,-4.563414,-1.396578,2.393118,-4.072404,-11.534848,-7.329080
56010,0.484283,0.258486,0.343881,746.942684,970.136077,21.096741,1304.983776,0.041989,-0.005405,-264.086914,...,-0.296346,-0.754429,-2.792803,-0.712064,-1.803433,-0.136713,-0.459130,-2.737273,-2.639398,-0.286367
140620,0.374304,0.210807,0.597701,1841.234857,2251.707474,17.951269,3786.443771,0.074505,-0.049629,-232.239548,...,6.536223,6.709996,3.652260,5.759178,3.540885,5.617452,2.707611,4.421750,0.538695,5.271423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88848,0.261277,0.185469,0.252963,1570.350022,1878.870124,25.783382,3138.606021,0.068279,0.035214,-169.085739,...,-8.436057,-0.776533,-11.080791,-4.695565,-7.673978,1.389265,-12.908194,-7.422032,-7.328937,3.733990
125826,0.284748,0.193866,0.329721,906.208063,1156.641992,23.763563,1782.472555,0.044039,-0.026723,-232.488602,...,-6.544378,-3.096047,-3.843890,-2.717676,-5.985793,-7.134715,-5.396871,1.757955,-0.397482,-1.932373
17644,0.650688,0.282871,0.468271,2013.292355,2046.622893,17.423149,3870.426596,0.102393,0.010657,-135.418427,...,-3.613782,1.983245,-5.530613,1.411764,-5.906091,-0.288435,-5.114183,0.673026,-3.388882,5.265758


In [26]:
1200/6397

0.18758793184305142

In [27]:
X_train,X_val, y_train, y_val = train_test_split(X_train,y_train, stratify=y_train,test_size=0.18)

In [28]:
encoder = LabelEncoder()

In [100]:
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)
y_val = encoder.transform(y_val)

In [101]:
scaler = StandardScaler()